<a href="https://colab.research.google.com/github/ambideXtrous9/T5-FineTuned-Model-for-NewsQA/blob/main/SentenceTransformer2DF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --quiet sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 95.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.9 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import torch
from sentence_transformers import SentenceTransformer, util

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = '/content/drive/MyDrive/MTP CODE/NewsQA_SPAN.feather'

In [5]:
df = pd.read_feather(path)
df

,question,answer,ans_pos,paragraph,answer_start,answer_end
0,Who is the managing director of Synergee Capital?,Vikram Dalal,"[133, 145]","""Investors can use a combination of governmen...",133,145
1,What is the yield of 30- and 40-year governmen...,7%,"[565, 567]","""Investors can use a combination of governmen...",565,567
2,What is the name of the ETF 2027 that a conser...,SDL,"[209, 212]","According to financial planners, an example o...",209,212
3,When would a conservative fixed income investo...,2027,"[217, 221]","According to financial planners, an example o...",217,221
4,What year would a conservative fixed income in...,2040,"[260, 264]","According to financial planners, an example o...",260,264
...,...,...,...,...,...,...
481753,When does Uncle Sam reopen for fully vaccinate...,November 8,"[295, 305]",NEW DELHI: This could be the last expansion of...,295,305
481754,When will there be three more weekly flights b...,from second week of November,"[116, 144]",It currently has 23 weekly flights to America....,116,144
481755,What type of 777s would have helped AI have mo...,Boeing,"[306, 312]",It currently has 23 weekly flights to America....,306,312
481756,What was the first wave of AI nonstops?,second,"[11, 17]","Before the second wave this summer, AI had abo...",11,17


In [6]:
df = df.iloc[:50000]

In [7]:
df1 = df.iloc[:30000].reset_index(drop=True)
df2 = df.iloc[30000:].reset_index(drop=True)

In [8]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [9]:
model.max_seq_length

384

In [10]:
corpus_embeddings = model.encode(df2['question'],show_progress_bar=True,convert_to_tensor=True)

Batches:   0%|          | 0/625 [00:00<?, ?it/s]

In [11]:
corpus_embeddings2 = model.encode(df1['question'],show_progress_bar=True,convert_to_tensor=True)

Batches:   0%|          | 0/938 [00:00<?, ?it/s]

In [12]:
type(corpus_embeddings)

torch.Tensor

In [13]:
corpus_embeddings.shape

torch.Size([20000, 768])

In [14]:
combined_tensor = torch.cat((corpus_embeddings, corpus_embeddings2), dim=0)

In [15]:
combined_tensor

tensor([[ 0.0564,  0.0136, -0.0270,  ..., -0.0107, -0.0511, -0.0403],
        [ 0.0771, -0.0201, -0.0034,  ..., -0.0344, -0.0329, -0.0340],
        [ 0.0454,  0.0327, -0.0212,  ..., -0.0019, -0.0842, -0.0245],
        ...,
        [-0.0127, -0.0356, -0.0145,  ..., -0.0228, -0.0172, -0.0142],
        [ 0.0433, -0.0753,  0.0055,  ..., -0.0242, -0.0341, -0.0062],
        [ 0.0287, -0.1074, -0.0014,  ...,  0.0050, -0.0388, -0.0047]],
       device='cuda:0')

In [16]:
idx = 0
question = df.iloc[idx]['question']
question

'Who is the managing director of Synergee Capital?'

In [17]:
combined_df = pd.concat([df2, df1])

# reset the index of the combined DataFrame
combined_df = combined_df.reset_index(drop=True)

In [18]:
question_embedding = model.encode(question,convert_to_tensor=True)

In [19]:
result = util.semantic_search(question_embedding,combined_tensor)[0]
result

[{'corpus_id': 17698, 'score': 1.0000001192092896},
 {'corpus_id': 17700, 'score': 1.0000001192092896},
 {'corpus_id': 20000, 'score': 1.0},
 {'corpus_id': 29505, 'score': 0.9756023287773132},
 {'corpus_id': 29509, 'score': 0.9756023287773132},
 {'corpus_id': 8240, 'score': 0.8094788789749146},
 {'corpus_id': 8241, 'score': 0.8094788789749146},
 {'corpus_id': 4395, 'score': 0.7935486435890198},
 {'corpus_id': 4402, 'score': 0.7935486435890198},
 {'corpus_id': 19378, 'score': 0.785905659198761}]

In [20]:
for item in result:
    print(round(item['score'],2),combined_df.iloc[item['corpus_id']]['paragraph'])

1.0  Bond dealers said yields can spike in the range of 25-100 basis points. A basis point is 0.01 percentage point. Depending on the quantum of selling, yields on bank perpetual bonds could surge 50-100 basis points, said Vikram Dalal, managing director, Synergee Capital. The spike will be even higher in the case of NBFC perpetual bonds, he said. “Smart investors could well sneak in, tapping arbitrage opportunities,” Manglunia said.
1.0  Bond dealers said yields can spike in the range of 25-100 basis points. A basis point is 0.01 percentage point. Depending on the quantum of selling, yields on bank perpetual bonds could surge 50-100 basis points, said Vikram Dalal, managing director, Synergee Capital. The spike will be even higher in the case of NBFC perpetual bonds, he said. “Smart investors could well sneak in, tapping arbitrage opportunities,” Manglunia said.
1.0  "Investors can use a combination of government bonds and low expense passive debt funds to build a ladder for their por

# **Accuaracy**

In [26]:
def predict_context(question):
    question_embedding = model.encode(question,convert_to_tensor=True)
    result = util.semantic_search(question_embedding,combined_tensor)[0][0]
    top_context = combined_df.iloc[result['corpus_id']]['paragraph']
    return top_context

In [27]:
def pred(idx):
    ques = df.iloc[idx]['question']
    print("Q : ", ques)
    predicted_context = predict_context(ques)
    print("Actual Context : ", combined_df.iloc[idx]['paragraph'])
    print("Predicted Context : ", predicted_context)

In [28]:
predict = df['question'].apply(predict_context)

In [29]:
res = (predict == df['paragraph']).value_counts()
res

True     48999
False     1001
dtype: int64

In [30]:
accuracy = (res[True] / len(predict))*100
accuracy

97.99799999999999

In [31]:
predict = df1['question'].apply(predict_context)
res = (predict == df1['paragraph']).value_counts()
accuracy = (res[True] / len(predict))*100
accuracy

97.51333333333334

In [32]:
predict = df2['question'].apply(predict_context)
res = (predict == df2['paragraph']).value_counts()
accuracy = (res[True] / len(predict))*100
accuracy

98.725